In [28]:
import pandas as pd
import requests as rq
import json

In [16]:


# This is the user agent string of Microsoft Edge. It's to trick the website that these requests are coming from old people who can't use computers!
header_data= { "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14931"}

<Response [400]>


In [37]:
season=[ "2012-13","2013-14","2014-15","2015-16","2016-17","2017-18","2018-19"]
home_away=["Home" , "Road" ]
#i is the index for season
i=0
#j is the index for home_away
j=0

urlliteral= f"https://stats.nba.com/stats/leaguedashteamstats?Conference=&DateFrom=&DateTo=&Division=&GameScope=&GameSegment=&LastNGames=0&LeagueID=00&Location={home_away[j]}&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season[i]}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision="

new=rq.get(urlliteral, headers=header_data)




In [38]:
new.json()

{'resource': 'leaguedashteamstats',
 'parameters': {'MeasureType': 'Base',
  'PerMode': 'Totals',
  'PlusMinus': 'N',
  'PaceAdjust': 'N',
  'Rank': 'N',
  'LeagueID': '00',
  'Season': '2012-13',
  'SeasonType': 'Regular Season',
  'PORound': 0,
  'Outcome': None,
  'Location': 'Home',
  'Month': 0,
  'SeasonSegment': None,
  'DateFrom': None,
  'DateTo': None,
  'OpponentTeamID': 0,
  'VsConference': None,
  'VsDivision': None,
  'TeamID': 0,
  'Conference': None,
  'Division': None,
  'GameSegment': None,
  'Period': 0,
  'ShotClockRange': None,
  'LastNGames': 0,
  'GameScope': None,
  'PlayerExperience': None,
  'PlayerPosition': None,
  'StarterBench': None,
  'TwoWay': 0},
 'resultSets': [{'name': 'LeagueDashTeamStats',
   'headers': ['TEAM_ID',
    'TEAM_NAME',
    'GP',
    'W',
    'L',
    'W_PCT',
    'MIN',
    'FGM',
    'FGA',
    'FG_PCT',
    'FG3M',
    'FG3A',
    'FG3_PCT',
    'FTM',
    'FTA',
    'FT_PCT',
    'OREB',
    'DREB',
    'REB',
    'AST',
    'TOV',
